In [1]:
import time
import datetime
from pathlib import Path
from collections import Counter
import random

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from scipy import ndimage

from model import conv_block
from data import example_to_tensor, normalize, add_channel_axis, train_test_split
from plot import plot_slice, plot_animated_volume, grid_plot_slices
from config import data_root_dir, seed

%matplotlib inline
plt.rcParams["figure.figsize"] = [15, 7]

In [23]:
tf.random.set_seed(seed)

input_shape = (48, 256, 256, 1)
# neg_tfrecord_glob = "covid-neg/*.tfrecord"
# pos_tfrecord_glob = "covid-pos/*.tfrecord"
neg_tfrecord_glob = "CT-[0-1]/*.tfrecord"
pos_tfrecord_glob = "CT-[3-4]/*.tfrecord"

epochs = 1000
patience = 20
batch_size = 8
learning_rate = 0.0001
dropout_rate = 0.5
val_perc = 0.12  # percentage from the already splitted training test
test_perc = 0.1

In [12]:
neg_tfrecord_fnames = [str(p) for p in Path(data_root_dir).glob(neg_tfrecord_glob)]
neg_x = (
    tf.data.TFRecordDataset(neg_tfrecord_fnames)
    .map(example_to_tensor, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(add_channel_axis, num_parallel_calls=tf.data.experimental.AUTOTUNE)
)
# num_neg = sum(1 for _ in neg_x)
num_neg = 938  # CT-0 + CT-1
# num_neg = 250
# num_neg = 254
print(f"Number of negative samples: {num_neg}")
neg_x

Number of negative samples: 938


<ParallelMapDataset shapes: (None, None, None, 1), types: tf.float32>

In [13]:
pos_tfrecord_fnames = [str(p) for p in Path(data_root_dir).glob(pos_tfrecord_glob)]
pos_x = (
    tf.data.TFRecordDataset(pos_tfrecord_fnames)
    .map(example_to_tensor, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(add_channel_axis, num_parallel_calls=tf.data.experimental.AUTOTUNE)
)
# num_pos = sum(1 for _ in pos_x)
num_pos = 47  # CT-3 + CT-4
# num_pos = 250
# num_pos = 856
print(f"Number of positive samples: {num_pos}")
pos_x

Number of positive samples: 47


<ParallelMapDataset shapes: (None, None, None, 1), types: tf.float32>

In [14]:
neg_y = tf.data.Dataset.from_tensors(tf.constant([0], dtype=tf.int8)).repeat(num_neg)
neg_dataset = tf.data.Dataset.zip((neg_x, neg_y))
neg_dataset

<ZipDataset shapes: ((None, None, None, 1), (1,)), types: (tf.float32, tf.int8)>

In [15]:
pos_y = tf.data.Dataset.from_tensors(tf.constant([1], dtype=tf.int8)).repeat(num_pos)
pos_dataset = tf.data.Dataset.zip((pos_x, pos_y))
pos_dataset

<ZipDataset shapes: ((None, None, None, 1), (1,)), types: (tf.float32, tf.int8)>

In [16]:
@tf.function
def random_rotate(volume, label):
    "Rotate the volume by a random degree"

    def scipy_rotate(volume):
        angle = tf.random.uniform(shape=(1,), minval=-180, maxval=180, dtype=tf.int32)[0].numpy()
        volume = ndimage.rotate(volume, angle, axes=(1, 2), reshape=False)
        volume[volume < 0] = 0
        volume[volume > 1] = 1
        return volume

    augmented_volume = tf.numpy_function(scipy_rotate, [volume], tf.float32)
    return augmented_volume, label

In [17]:
@tf.function
def random_shift(volume, label):
    "Shift the volume by a few random pixels"

    def scipy_shift(volume):
        shift_y, shift_x = tf.random.uniform(shape=(2,), minval=0, maxval=30, dtype=tf.int32).numpy()
        volume = ndimage.shift(volume, (0, shift_y, shift_x, 0))
        return volume

    augmented_volume = tf.numpy_function(scipy_shift, [volume], tf.float32)
    return augmented_volume, label

In [18]:
@tf.function
def random_contrast(volume, label):
    mean = tf.reduce_mean(volume)
    contrast_factor = tf.random.uniform(shape=(1,), minval=0, maxval=1, dtype=tf.float32)
    augmented_volume = (volume - mean) * contrast_factor + mean
    augmented_volume = tf.clip_by_value(augmented_volume, 0, 1)
    return augmented_volume, label

In [19]:
dataset = neg_dataset.concatenate(pos_dataset)
dataset, test_dataset = train_test_split(
    dataset,
    test_perc=test_perc,
    cardinality=(num_pos + num_neg),
    seed=seed,
)
test_dataset = test_dataset.batch(1)
train_dataset, val_dataset = train_test_split(
    dataset,
    test_perc=val_perc,
    cardinality=None,
    seed=seed,
)
val_dataset = (
    val_dataset.batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
)
train_dataset = (
    train_dataset.map(random_rotate, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(random_shift, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .batch(batch_size)
    .cache()  # must be called before shuffle
    .shuffle(buffer_size=64, reshuffle_each_iteration=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)
train_dataset

<PrefetchDataset shapes: (<unknown>, (None, 1)), types: (tf.float32, tf.int8)>

In [20]:
def count_labels(dataset):
    "Return a dictionary of the label count."
    return dict(Counter(label.numpy()[0] for _, label in dataset.unbatch()))


print(f"Train labels:\n\t{count_labels(train_dataset)}")
print(f"Validation labels:\n\t{count_labels(val_dataset)}")
print(f"Test labels:\n\t{count_labels(test_dataset)}")

Train labels:
	{0: 751, 1: 30}
Validation labels:
	{0: 96, 1: 10}
Test labels:
	{0: 91, 1: 7}


In [27]:
autoencoder = keras.models.load_model("models/autoencoder-20201029-125142.h5")
encoder = autoencoder.get_layer("encoder")
encoder.summary()
autoencoder.get_layer("decoder").summary()
autoencoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 256, 256, 1)] 0         
_________________________________________________________________
conv3d (Conv3D)              (None, 48, 256, 256, 32)  896       
_________________________________________________________________
alpha_dropout (AlphaDropout) (None, 48, 256, 256, 32)  0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 24, 128, 128, 32)  0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 24, 128, 128, 64)  55360     
_________________________________________________________________
alpha_dropout_1 (AlphaDropou (None, 24, 128, 128, 64)  0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 12, 64, 64, 64)    0   

In [ ]:
original, y = next(iter(train_dataset.unbatch().batch(1)))
print(f"label: {y}")
encoder_out = autoencoder.get_layer("encoder")(original, training=False)
decoder_out = autoencoder.get_layer("decoder")(encoder_out, training=False)

In [ ]:
plot_animated_volume(original[0, :], fps=5)

In [ ]:
z_index = 20
fig, ax = plt.subplots(ncols=3)
plot_slice(original[0, :], z_index, ax[0])
plot_slice(encoder_out[0, :], 0, ax[1])
plot_slice(decoder_out[0, :], z_index, ax[2])

In [28]:
encoder.trainable = False
cnn = keras.Sequential(
    [
        encoder,
        keras.layers.Flatten(),
        keras.layers.Dense(
            256, activation="relu",
        ),
        keras.layers.Dropout(dropout_rate),
        keras.layers.Dense(1, activation="sigmoid"),
    ],
    name="3dcnn",
)
cnn.summary()

Model: "3dcnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder (Functional)         (None, 6, 32, 32, 128)    277568    
_________________________________________________________________
flatten_2 (Flatten)          (None, 786432)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               201326848 
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 201,604,673
Trainable params: 201,327,105
Non-trainable params: 277,568
_________________________________________________________________


In [29]:
cnn.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[
        keras.metrics.TruePositives(name="tp"),
        keras.metrics.FalsePositives(name="fp"),
        keras.metrics.TrueNegatives(name="tn"),
        keras.metrics.FalseNegatives(name="fn"),
        keras.metrics.BinaryAccuracy(name="accuracy"),
        keras.metrics.Precision(name="precision"),
        keras.metrics.Recall(name="recall"),
        keras.metrics.AUC(name="auc"),
    ],
)

In [30]:
monitor_metric = "val_auc"

start_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
best_checkpoint = f"models/tl-{start_time}.h5"
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    best_checkpoint, monitor=monitor_metric, mode="max", verbose=1, save_best_only=True
)
early_stopping_cb = keras.callbacks.EarlyStopping(
    monitor=monitor_metric, patience=patience, mode="max"
)
log_dir = f"logs/tl-{start_time}"
file_writer = tf.summary.create_file_writer(log_dir)
with file_writer.as_default():
    tf.summary.text(
        "Hyperparameters",
        f"{seed=}; "
        f"{input_shape=}; "
        f"{epochs=}; "
        f"{patience=}; "
        f"{batch_size=}; "
        f"{learning_rate=}; "
        f"{dropout_rate=}; "
        f"{val_perc=}; "
        f"{test_perc=}",
        step=0,
    )
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    write_graph=False,
    profile_batch=0,
)
cnn.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
    callbacks=[checkpoint_cb, early_stopping_cb, tensorboard_cb],
)
cnn = keras.models.load_model(best_checkpoint)

Epoch 1/1000
     98/Unknown - 15s 152ms/step - loss: 4.8087 - tp: 2.0000 - fp: 27.0000 - tn: 724.0000 - fn: 28.0000 - accuracy: 0.9296 - precision: 0.0690 - recall: 0.0667 - auc: 0.5087
Epoch 00001: val_auc improved from -inf to 0.50000, saving model to models/tl-20201104-194055.h5
98/98 [==============================] - 250s 3s/step - loss: 4.8087 - tp: 2.0000 - fp: 27.0000 - tn: 724.0000 - fn: 28.0000 - accuracy: 0.9296 - precision: 0.0690 - recall: 0.0667 - auc: 0.5087 - val_loss: 2.8985 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 96.0000 - val_fn: 10.0000 - val_accuracy: 0.9057 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5000
Epoch 2/1000
98/98 [==============================] - ETA: 0s - loss: 1.1582 - tp: 3.0000 - fp: 25.0000 - tn: 726.0000 - fn: 27.0000 - accuracy: 0.9334 - precision: 0.1071 - recall: 0.1000 - auc: 0.5123
Epoch 00002: val_auc did not improve from 0.50000
98/98 [==============================] - 34s 342ms/step - loss: 1.1582 - tp: 

98/98 [==============================] - ETA: 0s - loss: 0.1644 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 751.0000 - fn: 30.0000 - accuracy: 0.9616 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.8506
Epoch 00013: val_auc did not improve from 0.56302
98/98 [==============================] - 32s 323ms/step - loss: 0.1644 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 751.0000 - fn: 30.0000 - accuracy: 0.9616 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.8506 - val_loss: 0.4530 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 96.0000 - val_fn: 10.0000 - val_accuracy: 0.9057 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.4849
Epoch 14/1000
98/98 [==============================] - ETA: 0s - loss: 0.1711 - tp: 1.0000 - fp: 0.0000e+00 - tn: 751.0000 - fn: 29.0000 - accuracy: 0.9629 - precision: 1.0000 - recall: 0.0333 - auc: 0.8503
Epoch 00014: val_auc did not improve from 0.56302
98/98 [==============================] - 32s 327ms/step - loss: 0.1711 - tp: 1.0000 -

98/98 [==============================] - ETA: 0s - loss: 0.1160 - tp: 1.0000 - fp: 0.0000e+00 - tn: 751.0000 - fn: 29.0000 - accuracy: 0.9629 - precision: 1.0000 - recall: 0.0333 - auc: 0.9237
Epoch 00025: val_auc did not improve from 0.59063
98/98 [==============================] - 32s 325ms/step - loss: 0.1160 - tp: 1.0000 - fp: 0.0000e+00 - tn: 751.0000 - fn: 29.0000 - accuracy: 0.9629 - precision: 1.0000 - recall: 0.0333 - auc: 0.9237 - val_loss: 0.4780 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 96.0000 - val_fn: 10.0000 - val_accuracy: 0.9057 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5047
Epoch 26/1000
98/98 [==============================] - ETA: 0s - loss: 0.1024 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 751.0000 - fn: 30.0000 - accuracy: 0.9616 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.9388
Epoch 00026: val_auc did not improve from 0.59063
98/98 [==============================] - 35s 358ms/step - loss: 0.1024 - tp: 0.0000e+00 - fp: 0.0

98/98 [==============================] - 32s 324ms/step - loss: 0.1167 - tp: 1.0000 - fp: 0.0000e+00 - tn: 751.0000 - fn: 29.0000 - accuracy: 0.9629 - precision: 1.0000 - recall: 0.0333 - auc: 0.9185 - val_loss: 0.4716 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 96.0000 - val_fn: 10.0000 - val_accuracy: 0.9057 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5396
Epoch 38/1000
98/98 [==============================] - ETA: 0s - loss: 0.1224 - tp: 1.0000 - fp: 0.0000e+00 - tn: 751.0000 - fn: 29.0000 - accuracy: 0.9629 - precision: 1.0000 - recall: 0.0333 - auc: 0.9141
Epoch 00038: val_auc did not improve from 0.59063
98/98 [==============================] - 35s 358ms/step - loss: 0.1224 - tp: 1.0000 - fp: 0.0000e+00 - tn: 751.0000 - fn: 29.0000 - accuracy: 0.9629 - precision: 1.0000 - recall: 0.0333 - auc: 0.9141 - val_loss: 0.5632 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 96.0000 - val_fn: 10.0000 - val_accuracy: 0.9057 - val_precision: 0.0000e+00 - va

In [ ]:
cnn = keras.models.load_model("models/tl-20201104-150014.h5")
cnn.evaluate(val_dataset, verbose=1, return_dict=True)

In [ ]:
cnn.trainable = True
patience = 30
learning_rate = 0.000001

In [ ]:
x = next(iter(train_dataset.unbatch().batch(1)))
with tf.GradientTape() as tape:
    last_conv_layer = cnn.get_layer("encoder").get_layer("conv3d_19")
    iterate = tf.keras.models.Model([cnn.inputs], [cnn.output, last_conv_layer.output])
    model_out, last_conv_layer = iterate(x)
    class_out = model_out[:, np.argmax(model_out[0])]
    grads = tape.gradient(class_out, last_conv_layer)
    pooled_grads = keras.backend.mean(grads, axis=(0, 1, 2))
  
heatmap = tf.reduce_mean(tf.multiply(pooled_grads, last_conv_layer), axis=-1)

In [ ]:
cnn.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[
        # keras.metrics.TruePositives(name="tp"),
        # keras.metrics.FalsePositives(name="fp"),
        # keras.metrics.TrueNegatives(name="tn"),
        # keras.metrics.FalseNegatives(name="fn"),
        keras.metrics.BinaryAccuracy(name="accuracy"),
        # keras.metrics.Precision(name="precision"),
        # keras.metrics.Recall(name="recall"),
        # keras.metrics.AUC(name="auc"),
    ],
)

In [ ]:
monitor_metric = "val_accuracy"

start_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
best_checkpoint = f"models/tl-finetuning-{start_time}.h5"
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    best_checkpoint, monitor=monitor_metric, mode="max", verbose=1, save_best_only=True
)
early_stopping_cb = keras.callbacks.EarlyStopping(
    monitor=monitor_metric, patience=patience, mode="max"
)
log_dir = f"logs/tl-finetuning-{start_time}"
file_writer = tf.summary.create_file_writer(log_dir)
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    write_graph=False,
    profile_batch=0,
)
cnn.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
    callbacks=[checkpoint_cb, early_stopping_cb, tensorboard_cb],
)
with file_writer.as_default():
    tf.summary.text(
        "Hyperparameters",
        f"{seed=}; "
        f"{input_shape=}; "
        f"{epochs=}; "
        f"{patience=}; "
        f"{batch_size=}; "
        f"{learning_rate=}; "
        f"{dropout_rate=}; "
        f"{val_perc=}; "
        f"{test_perc=}",
        step=0,
    )
cnn = keras.models.load_model(best_checkpoint)

In [ ]:
x, y = next(iter(val_dataset.unbatch().batch(1).skip(1)))
prediction = cnn(x, training=False)
print(f"real: {y.numpy()}, prediction: {prediction.numpy()}")
plot_animated_volume(x[0, :], fps=2)